In [16]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)


In [25]:
orders = pd.read_csv(
    "data/orders.csv",
    dtype={
        "order_id": "int32",
        "user_id": "int32",
        "order_number": "int16",
        "order_dow": "int8",
        "order_hour_of_day": "int8"
    }
)

order_products= pd.read_csv('data/order_products__prior.csv',
dtype={
    "order_id": "int32",
    "user_id": "int32",
})

products= pd.read_csv('data/products.csv',
dtype={
    "product_id": "int32",
    'aisle_id': 'int16',
    'department_id': 'int16',
})

In [29]:
op_small = order_products.sample(2_000_000, random_state=42)
df = (
    op_small
    .merge(orders, on="order_id", how="left")
    .merge(products, on="product_id", how="left")
)
df.shape
df.head()
df.dtypes


order_id                    int32
product_id                  int64
add_to_cart_order           int64
reordered                   int64
user_id                     int32
eval_set                   object
order_number                int16
order_dow                    int8
order_hour_of_day            int8
days_since_prior_order    float64
product_name               object
aisle_id                    int16
department_id               int16
dtype: object

In [30]:
(df.isna()
   .mean()
   .mul(100)
   .round(2)
   .sort_values(ascending=False))



days_since_prior_order    6.39
product_id                0.00
add_to_cart_order         0.00
reordered                 0.00
order_id                  0.00
user_id                   0.00
eval_set                  0.00
order_dow                 0.00
order_number              0.00
order_hour_of_day         0.00
product_name              0.00
aisle_id                  0.00
department_id             0.00
dtype: float64

In [32]:
def winsorize(series, low=0.01, high=0.99):
    q_low = series.quantile(low)
    q_high = series.quantile(high)
    return series.clip(q_low, q_high)
num_cols = [
    "days_since_prior_order",
    "add_to_cart_order"
]

for col in num_cols:
    if col in df.columns:
        df[col] = winsorize(df[col])


In [35]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
order_id,2000000.0,NaN,NaN,NaN,1712692.586495,987176.180294,2.0,858471.0,1714216.0,2566657.25,3421083.0
product_id,2000000.0,NaN,NaN,NaN,25564.82708,14101.559138,1.0,13517.0,25237.0,37915.0,49688.0
add_to_cart_order,2000000.0,NaN,NaN,NaN,8.29545,6.824239,1.0,3.0,6.0,11.0,33.0
reordered,2000000.0,NaN,NaN,NaN,0.589659,0.491896,0.0,0.0,1.0,1.0,1.0
user_id,2000000.0,NaN,NaN,NaN,102975.836816,59444.001704,1.0,51520.0,102635.0,154411.0,206209.0
eval_set,2000000,1,prior,2000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
order_number,2000000.0,NaN,NaN,NaN,17.138831,17.51165,1.0,5.0,11.0,24.0,99.0
order_dow,2000000.0,NaN,NaN,NaN,2.737325,2.089432,0.0,1.0,3.0,5.0,6.0
order_hour_of_day,2000000.0,NaN,NaN,NaN,13.427357,4.24799,0.0,10.0,13.0,16.0,23.0
days_since_prior_order,1872229.0,NaN,NaN,NaN,11.106105,8.784899,0.0,5.0,8.0,15.0,30.0
